In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
import sys
sys.path.append("/Workspace/Users/mayur10594@gmail.com/ETL_project")
from config.utils import get_logger
logger = get_logger("dataloadinitiate")

In [0]:
import json, yaml, os
from config.utils import load_config,create_tables,load_all_schemas

In [0]:
dbutils.widgets.text("fileList", "")
dbutils.widgets.text("taxYear", "")
dbutils.widgets.text("clientId", "")
dbutils.widgets.text("env","")
dbutils.widgets.text("bronze_path", "")
dbutils.widgets.text("maps","")
fileList = dbutils.widgets.get("fileList")
taxYear = dbutils.widgets.get("taxYear")
clientId = dbutils.widgets.get("clientId")
env = dbutils.widgets.get("env")
bronze_path = dbutils.widgets.get("bronze_path")
maps=json.loads(dbutils.widgets.get("maps"))
logger.info(f"fileList: {fileList}, taxYear: {taxYear}, clientId: {clientId}, env: {env}, bronze_path: {bronze_path}, maps to be loaded: {maps}")

In [0]:
# Dictionary to hold DataFrames
dfs = {}

for map_name in maps:
    # Build CSV path
    # csv_path = f"{in_path}/{map_name}_data.csv"
    # parquet_path=f"{bronze_path}/{map_name}_data/"
    # Evaluate schema string (assumes map_name_schema exists as a variable)
    #schema = schemas_map[f"{map_name}.{map_name}"]
    # Read CSV into DataFrame
    #transformations = pipeline_config[map_name]
    table=f"cp_database.{clientId}_{map_name}"
    df = spark.read.table(table)
    #df = spark.read.csv(csv_path, schema=schema, header=True)
    # df = df.transform(add_audit_cols(df,f"{map_name}_data"))
    # Store in dictionary
    dfs[map_name] = df
    #print(f"df_{map_name} is created and stored in dfs['{map_name}']")
    logger.info(f"df_{map_name} is created and stored in dfs['{map_name}']")

In [0]:
# dfs['cards'].display()
dfs['cards'].where('card_type=="Debit"').count()

In [0]:
dfs['transactions']=dfs['transactions'].withColumn('date',to_date('date','dd-MM-yyyy HH:mm')).withColumn('amount',regexp_replace('amount','[$]','').cast('decimal(10,2)'))

In [0]:
dfs['transactions'].display()

In [0]:
dfs['transactions'].where('errors is not Null').count()